In [1]:
from src.constants import *
from src.model.my_model import MyModel

In [2]:
model = MyModel(in_channels=1, out_channels=3, embed_dim=64, skip_transformer=False, transformer_channels=4).to(DEVICE)


In [3]:
example = torch.rand(size=(4, 1, 128, 128, 128)).cuda()

In [4]:
%%time
model(example)

CPU times: user 1.68 s, sys: 996 ms, total: 2.67 s
Wall time: 2.67 s


tensor([[[[[-5.3307e-02,  4.6436e-01,  3.6252e-01,  ..., -2.1503e-01,
             1.0333e+00,  1.0682e+00],
           [ 5.2256e-01,  1.1752e-01,  4.3112e-01,  ...,  4.0810e-01,
            -8.2436e-02, -2.3305e-02],
           [-2.2785e-01,  3.5643e-01,  1.4694e+00,  ...,  6.3092e-01,
             1.6623e+00,  4.0282e-01],
           ...,
           [ 5.3539e-01,  2.0179e+00,  1.3218e-01,  ...,  2.2324e-01,
            -1.0928e-01,  1.3928e-01],
           [-8.7672e-02, -1.4946e-01, -2.7511e-01,  ..., -6.9575e-02,
             3.3178e-01,  3.8846e-01],
           [ 5.3311e-01, -2.1665e-01,  1.2957e-01,  ..., -4.0477e-02,
            -6.8603e-02,  2.1484e-01]],

          [[ 9.5992e-01,  5.9080e-01,  1.1695e+00,  ..., -1.7078e-01,
             1.4446e+00,  3.6953e-01],
           [ 6.3880e-01, -1.7118e-01, -4.6627e-02,  ...,  7.1861e-01,
            -5.9007e-02,  7.2032e-01],
           [ 4.4927e-01,  1.9451e+00,  1.1199e+00,  ...,  7.3720e-01,
             4.0532e-01,  4.2122e-01],
 